In [1]:
import pandas as pd
import numpy as np
import pingouin as pg
import requests
import datetime

In [99]:
def read_csv(url, sep=','):
    key = url
    api_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key="+key
    response = requests.get(api_url)
    download_url = response.json()['href']
    df = pd.read_csv(download_url, sep=sep)
    return df

In [100]:
problem1_reg = read_csv('https://disk.yandex.ru/d/Xn78-_wgoIT6dA', sep = ';')
problem1_auth = read_csv('https://disk.yandex.ru/d/BxnJZSKyNVQt_A', sep = ';')

In [101]:
total_users = pd.merge(left = problem1_auth, right=problem1_reg, on ='uid', how='left')

In [102]:
total_users.auth_ts = total_users.auth_ts.apply(lambda x: datetime.datetime.utcfromtimestamp(x))
total_users.reg_ts = total_users.reg_ts.apply(lambda x: datetime.datetime.utcfromtimestamp(x))

In [103]:
def retention(date_reg_1="1998-11-18", date_reg_2="2020-09-23"):

    total_users_date = total_users.query('(reg_ts >= @date_reg_1 & reg_ts <= @date_reg_2) & \
                                          (auth_ts >= @date_reg_1 & auth_ts <= @date_reg_2)')
    total_users_date['day_reg'] = pd.to_datetime(total_users_date.reg_ts.dt.date)
    total_users_date['day_auth'] = pd.to_datetime(total_users_date.auth_ts.dt.date)
    total_users_date['timedelta'] = (total_users_date.day_auth - total_users_date.day_reg).dt.days

    cohort = total_users_date.groupby(['day_reg', 'timedelta'], as_index=False).uid.nunique()
    cohort_pivot = cohort.pivot(index = 'day_reg', columns = 'timedelta', values = 'uid')
    cohort_size = cohort_pivot[0]
    retention = cohort_pivot.divide(cohort_size, axis=0)
    return retention

In [105]:
retention('2019-01-01', '2019-01-31')

/var/folders/br/yryz75s94419__57gscfgy900000gn/T/ipykernel_29249/3367238425.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_users_date['day_reg'] = pd.to_datetime(total_users_date.reg_ts.dt.date)
/var/folders/br/yryz75s94419__57gscfgy900000gn/T/ipykernel_29249/3367238425.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_users_date['day_auth'] = pd.to_datetime(total_users_date.auth_ts.dt.date)
/var/folders/br/yryz75s94419__57gscfgy900000gn/T/ipykernel_29249/3367238425.py:7: SettingWithCopyW

timedelta,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
day_reg,,,,,,,,,,,,,,,,,,,,,
2019-01-01,1.0,0.020583,0.037736,0.049743,0.044597,0.063465,0.082333,0.058319,0.054889,0.060034,...,0.029160,0.054889,0.030875,0.042882,0.037736,0.032590,0.042882,0.036021,0.041166,0.02916
2019-01-02,1.0,0.017153,0.049743,0.036021,0.049743,0.080617,0.054889,0.070326,0.030875,0.041166,...,0.041166,0.036021,0.039451,0.036021,0.022298,0.032590,0.024014,0.039451,0.018868,NaN
2019-01-03,1.0,0.017094,0.030769,0.044444,0.059829,0.063248,0.064957,0.058120,0.022222,0.049573,...,0.044444,0.034188,0.035897,0.023932,0.037607,0.042735,0.032479,0.042735,NaN,NaN
2019-01-04,1.0,0.027350,0.037607,0.042735,0.054701,0.054701,0.080342,0.052991,0.042735,0.052991,...,0.046154,0.042735,0.035897,0.032479,0.029060,0.044444,0.037607,NaN,NaN,NaN
2019-01-05,1.0,0.018771,0.040956,0.046075,0.049488,0.081911,0.071672,0.069966,0.059727,0.040956,...,0.046075,0.042662,0.052901,0.027304,0.047782,0.029010,NaN,NaN,NaN,NaN
2019-01-06,1.0,0.017007,0.037415,0.052721,0.047619,0.059524,0.056122,0.057823,0.052721,0.047619,...,0.037415,0.027211,0.039116,0.035714,0.039116,NaN,NaN,NaN,NaN,NaN
2019-01-07,1.0,0.017007,0.042517,0.040816,0.054422,0.073129,0.073129,0.066327,0.054422,0.054422,...,0.042517,0.039116,0.042517,0.035714,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-08,1.0,0.030560,0.040747,0.050934,0.057725,0.071307,0.071307,0.066214,0.057725,0.039049,...,0.028862,0.044143,0.037351,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-09,1.0,0.010169,0.027119,0.042373,0.052542,0.059322,0.064407,0.044068,0.057627,0.050847,...,0.027119,0.040678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
